In [2]:
'''Code to train the CY metric'''
# Import libraries
import numpy as np
import os
import sys
import yaml
import pickle
import tensorflow as tf

# Setup path for cymetric package  
import pathlib
import sys, pathlib

try:
    # When running as a script
    FILE = pathlib.Path(__file__).resolve()
except NameError:
    # When running in a Jupyter notebook
    FILE = pathlib.Path().resolve() / "notebook.ipynb"

_parent_dir = FILE.parent.parent.parent
_cymetric_dir = _parent_dir / "cymetric"

if str(_parent_dir) not in sys.path:
    sys.path.insert(0, str(_parent_dir))
if str(_cymetric_dir) not in sys.path:
    sys.path.insert(0, str(_cymetric_dir))

# Create alias to fix cymetric internal imports
# import cymetric
# if hasattr(cymetric, 'cymetric'):
#     sys.modules['cymetric'] = cymetric.cymetric

# Import cymetric functions
from cymetric.pointgen.pointgen import PointGenerator
# from cymetric.models.callbacks import SigmaCallback
# from cymetric.models.tfmodels import PhiFSModel
from cymetric.models.callbacks import RicciCallback, SigmaCallback, VolkCallback, KaehlerCallback, TransitionCallback
from cymetric.models.models import MultFSModel
# from cymetric.models.metrics import SigmaLoss
# from cymetric.models.tfhelper import prepare_tf_basis, train_model
from cymetric.models.helper import prepare_basis, train_model


/opt/homebrew/Caskroom/miniforge/base/envs/g2_ml/lib/python3.9/importlib/__init__.py:127: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _bootstrap._gcd_import(name[level:], package, level)


In [ ]:
# Generate the training data
monomials = 5*np.eye(5, dtype=np.int64)
coefficients = np.ones(5)
kmoduli = np.ones(1)
ambient = np.array([4])

pg = PointGenerator(monomials, coefficients, kmoduli, ambient)
#points = pg.generate_points(100)

# Save the training data
dirname = './train_data'
n_p = 200000
kappa = pg.prepare_dataset(n_p, dirname)
data = np.load(os.path.join(dirname, 'dataset.npz'))
pg.prepare_basis(dirname, kappa=kappa)

# Load the training data
data = np.load(os.path.join(dirname, 'dataset.npz'))
BASIS = np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True)
BASIS = prepare_basis(BASIS)



/opt/homebrew/Caskroom/miniforge/base/envs/g2_ml/lib/python3.9/importlib/__init__.py:127: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _bootstrap._gcd_import(name[level:], package, level)
/opt/homebrew/Caskroom/miniforge/base/envs/g2_ml/lib/python3.9/importlib/__init__.py:127: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _bootstrap._gcd_import(name[level:], package, level)
/opt/homebrew/Caskroom/miniforge/base/envs/g2_ml/lib/python3.9/importlib/__init__.py:127: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_r

In [10]:
rcb = RicciCallback((data['X_val'], data['y_val']), data['val_pullbacks'])
scb = SigmaCallback((data['X_val'], data['y_val']))
volkcb = VolkCallback((data['X_val'], data['y_val']))
kcb = KaehlerCallback((data['X_val'], data['y_val']))
tcb = TransitionCallback((data['X_val'], data['y_val']))
cb_list = [rcb, scb, kcb, tcb, volkcb]

In [17]:
nlayer = 3
nHidden = 64
act = 'gelu'
nEpochs = 2
bSizes = [64, 50000]
alpha = [1., 1., 1., 1., 1.]
nfold = 3
n_in = 2*5
n_out = nfold**2

In [13]:
nn = tf.keras.Sequential()
nn.add(tf.keras.Input(shape=(n_in,)))
for i in range(nlayer):
    nn.add(tf.keras.layers.Dense(nHidden, activation=act))
nn.add(tf.keras.layers.Dense(n_out, use_bias=False))

In [14]:
fmodel = MultFSModel(nn, BASIS, alpha=alpha)

In [16]:
opt = tf.keras.optimizers.Adam()

In [18]:
fmodel, training_history = train_model(fmodel, data, optimizer=opt, epochs=nEpochs, batch_sizes=bSizes, verbose=1, callbacks=cb_list)


Epoch  1/2
1503/2813 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - kaehler_loss: 0.0054 - loss: 0.2004 - ricci_loss: 0.0000e+00 - sigma_loss: 0.1812 - transition_loss: 0.0138 - volk_loss: 0.0000e+00

KeyboardInterrupt: 